<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">

## Required header files

In [ ]:
#include <iostream>
#include "TChain.h"
#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TSystem.h"
#include "TROOT.h"

#include "TMVA/Factory.h"
#include "TMVA/Tools.h"
#include "TMVA/DataLoader.h"

## Declare Factory and DataLoader

In [ ]:
TMVA::Tools::Instance();

TString outfileName( "TMVAOutputDL.root" );
TFile* outputFile = TFile::Open( outfileName, "RECREATE" );

TMVA::Factory *factory = new TMVA::Factory( "TMVAClassification", outputFile,                                       "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" );
   
TMVA::DataLoader *loader1=new TMVA::DataLoader("mymc-dataset1");
TMVA::DataLoader *loader2=new TMVA::DataLoader("mymc-dataset2");
TFile *input = TFile::Open( "MyMCdataset.root");                

## Adding variables to DataLoader

In [ ]:
//adding variables to dataset 1
loader1->AddVariable( "var0", 'F' );
loader1->AddVariable( "var1", 'F' );
loader1->AddVariable( "var2", 'F' );

//adding variables to dataset 2
loader2->AddVariable( "var0", 'F' );
loader2->AddVariable( "var1", 'F' );
loader2->AddVariable( "var2", 'F' );
loader2->AddVariable( "var3 := var0-var1", 'F' );
loader2->AddVariable( "var4 := var0*var2", 'F' );
loader2->AddVariable( "var5 := var1+var2", 'F' );

## Setting up dataset from Trees

In [ ]:
std::cout << "--- TMVAClassification       : Using input file: " << input->GetName() << std::endl;
   
// --- Register the training and test trees

TTree *tsignal     = (TTree*)input->Get("MyMCSig");
TTree *tbackground = (TTree*)input->Get("MyMCBkg");
   
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;

TCut mycuts = "";
TCut mycutb = "";
   
loader1->AddSignalTree    ( tsignal,     signalWeight     );
loader1->AddBackgroundTree( tbackground, backgroundWeight );       
loader1->fSignalWeight=signalWeight;
loader1->fBackgroundWeight=backgroundWeight;
loader1->fTreeS=tsignal;
loader1->fTreeB=tbackground;
loader1->PrepareTrainingAndTestTree( mycuts, mycutb,  "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V" ); 


loader2->AddSignalTree    ( tsignal,     signalWeight     );
loader2->AddBackgroundTree( tbackground, backgroundWeight );       
loader2->fSignalWeight=signalWeight;
loader2->fBackgroundWeight=backgroundWeight;
loader2->fTreeS=tsignal;
loader2->fTreeB=tbackground;
loader2->PrepareTrainingAndTestTree( mycuts, mycutb,  "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V" );   


// TMVA ANN: MLP (recommended ANN) -- all ANNs in TMVA are Multilayer Perceptrons
factory->BookMethod( loader2,TMVA::Types::kMLP, "MLP", "!V:NeuronType=tanh:VarTransform=N:NCycles=50:HiddenLayers=N+5:TestRate=5:!UseRegulator" );## Booking Methods using different DataLoaders

In [ ]:
// TMVA ANN: MLP Multilayer Perceptrons
factory->BookMethod(loader1,TMVA::Types::kMLP, "MLP", "!V:NeuronType=tanh:VarTransform=N:NCycles=150:HiddenLayers=N+5:TestRate=5:!UseRegulator" );

//Boosted Decision Trees
factory->BookMethod(loader2,TMVA::Types::kBDT, "BDT","!V:NTrees=5:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

## Training the Methods

In [ ]:
factory->TrainAllMethods();

## Testing and Evaluating the data

In [ ]:
factory->TestAllMethods();

factory->EvaluateAllMethods();    

// Save the output
outputFile->Close();

std::cout << "==> TMVAClassification is done!" << std::endl;

delete factory;
delete loader1;
delete loader2;

## Ploting ROC Curve

In [ ]:
TFile *file=TFile::Open("TMVAOutputDL.root");

gDirectory->cd("mymc-dataset1");
TMVA::plot_efficiencies(file,  2,gDirectory);
c->Draw();

In [ ]:
if(c) delete c;
file->cd();
gDirectory->cd("mymc-dataset2");
TMVA::plot_efficiencies(file,  2,gDirectory);
c->Draw();